# PID控制原理与增量式PID算法

> 本小节在理论上讲解 PID 控制原理, 先引出模拟 PID 控制公式, 然后离散化处理得到数字 PID 控制公式, 推导出位置式 PID 控制公式, 再进一步推导出增量式 PID 控制公式.

将偏差的比例(Proportion), 积分(Integral)和微分(Differential)通过线性组合构成控制量, 用这一控制量对被控对象进行控制, 这样的控制器称 PID 控制器.

## 模拟 PID 控制

在模拟控制系统中, 控制器最常用的控制规律是PID控制. 为了说明控制器的工作原理, 先看一个例子. 下图所示是一个小功率直流电机的调速框图:

![](https://c.miaowlabs.com/img/2019-01-03_210504.png)

上图中, 给定转速 $n_0(t)$ 与实际转速 $n(t)$ 进行比较, 其差值 $e(t) = n_0(t) - n(t)$, 经过 PID 控制器调整后输出电压控制信号 $u(t)$, $u(t)$ 经过功率放大后, 驱动直流电动机改变其转速.

> 强化:
>
> - PID 控制器能输出的是电压控制信号

抽象化的模拟PID控制系统原理框图, 如下图所示, 该系统由模拟PID控制器和被控对象组成:

![](https://c.miaowlabs.com/img/2019-01-03_210531.png)

图中 $r(t)$ 是给定预期输出值, $y(t)$ 是系统的实际输出值, 给定值与实际输出值构成控制偏差:

$$
e(t) = r(t) - y(t)
\tag{式1-1}
$$

$e(t)$ 作为 PID 控制的输入, $u(t)$ 作为 PID 控制器的输出和被控对象的输入. 所以模拟 PID 控制器的控制规律为:

$$
u(t) = K_p e(t) + K_i \int_{0}^{t} e(\tau)d\tau + K_d \frac{de(t)}{dt}
\tag{式1-2}
$$


其中

- $K_p$ 是比例增益，是调适参数
- $K_i$ 是积分增益，也是调适参数
- $K_d$ 是微分增益，也是调适参数
- $e$ 是误差, 值等于 设定值(SP) - 回授值(PV)
- $t$ 是目前时间
- $\tau$ 积分变数，数值从0到目前时间

用更专业的话来讲, PID控制器可以视为是频域系统的滤波器器. 在计算控制器最终是否会达到稳定结果时, 此性质很有用. 如果数值挑选不当, 控制系统的输入值会反复振荡, 这导致系统可能永远无法达到预设值.

PID 控制器的一般转移函数是:

$$
H(s) = \frac{K_d s^2 + K_p s + K_i}{s+C}
$$

其中 $C$ 是一个取决于系统带宽的常数.

## 数字PID控制

由于计算机的出现, 计算机进入了控制领域. 人们将模拟 PID 控制规律引入到计算机中来, 对 (式1-2) 的 PID 控制规律进行适当的变换, 就可以用软件实现 PID 控制, 即数字 PID 控制.

数字式 PID 控制算法可以分为位置式 PID 和增量式 PID 控制算法.

### 位置式PID算法

由于计算机控制是一种采样控制, 它只能根据采样时刻的偏差计算控制量, 而不能像模拟控制那样连续输出控制量进行连续控制. 由于这一特点, (式1-2)中的积分项和微分项不能直接使用, 必须进行离散化处理.

离散化处理的方法为: 以 $T$ 作为采样周期, $k$ 作为采样序号, 则离散采样时间 $kT$ 对应着连续时间 $t$, 用矩形法数值积分近似代替积分, 用一阶后向差分近似代替微分, 可作如下近似变换:

$$
\begin{align*}
    t \approx & k T \quad (k = 0, 1, 2, \dots) \\ \\
    \int_{0}^{t} e(t) dt \approx & T \sum_{j=0}^{k} e(jT) = T \sum_{j=0}{k} e_j \\ \\
    \frac{de(t)}{dt} \approx & \frac{e(kT) - e((k-1)T)}{T} = \frac{e_k - e_{k-1}}{T} \\
\end{align*}
$$


上式中为了表达的方便将类似于 $e(kT)$ 简化成 $e_k$ 等.

将 (式2-1) 代入 (式1-2), 就可以得到离散的 PID 表达式为:

$$
    u_k = K_p e_k + K_i \sum_{j=0}^{k} e_j + K_d(e_k - e_{k-1})
    \tag{式2-3}
$$

其中:

- $k$ 为采样序号, $k = 0,1,2,\dots$
- $u_k$ 为第 $k$ 次采样时刻的计算机输出值
- $e_k$ 为第 $k$ 次采样时刻输入的偏差值
- $e_k-1$ 为第 $k-1$ 次采样时刻输入的偏差值
- $K_i$ 为积分系数, $K_i = kp* \frac{T}{T_i}$
- $K_d$ 为微分系数, $K_d = K_p * \frac{Td}{T}$


> 注意
>
> (式2-3) 和 (式1-2) 里面的系数不完全等价, 这里只是在描述推导数字式 PID 的公式形式.

如果采样周期足够小, 则 (式2-3) 的近似计算可以获得足够精确的结果, 离散控制过程与连续过程十分接近.

(式2-3) 表示的控制算法式直接按 (式1-2) 所给出的 PID 控制规律定义进行计算的, 所以它给出了全部控制量的大小, 因此被称为全量式或位置式 PID 控制算法.

> 这句话在说啥???

这种算法的缺点是: 由于全量输出, 所以每次输出均与过去状态有关, 计算时要对 $e_k$ 进行累加, 工作量大. 并且, 因为计算机输出的对应的是执行机构的实际位置, 如果计算机出现故障, 输出将大幅度变化, 会引起执行机构的大幅度变化, 有可能因此造成严重的生产事故, 这在实际生产际中是不允许的.

### 增量式PID算法

所谓增量式 PID 是指数字控制器的输出只是控制量的增量 $\Delta{u_k}$, 当执行机构需要的控制量是增量, 而不是位置量的绝对数值时, 可以使用增量式 PID 控制算法进行控制.

增量式 PID 控制算法可以通过 (式2-3) 推导出:

$$
\begin{align*}
    \Delta{u_k} = & u_k - u_{k-1} \\ \\
                = & \left ( K_p e_k + K_i \sum_{j=0}^{k} e_j + K_d(e_k - e_{k-1}) \right ) -
                    \left ( K_p e_{k-1} + K_i \sum_{j=0}^{{k-1}} e_j + K_d(e_{k-1} - e_{k-2}) \right ) \\ \\
                = & K_p (e_k - e_{k-1}) + K_i e_k + K_d(e_k - 2 e_{k-1} + e_{k-2}) \\ \\
                = & (K_p + K_i + K_d) e_k - (K_p + 2K_d) e_{k-1} + K_d e_{k-2} \\ \\
                = & A e_k + B e_{k-1} + C e_{k-2}
\end{align*}

\tag{式 2-5}
$$

由 (式2-5) 可以看出, 如果计算机控制系统采用恒定的采样周期 $T$, 一旦确定 A, B, C, 只要使用前后三次测量的偏差值, 就可以由 (式2-5) 求出控制量.

增量式 PID 控制算法与位置式 PID 算法 (式2-2) 相比, 计算量小的多, 因此在实际中得到广泛的应用.

而位置式 PID 控制算法也可以通过增量式控制算法推出送推计算公式:

$$
u_k = u_{k - 1} + \Delta{u_k}
\tag{式2-6}
$$

(式2-6) 就是目前在计算机控制中广泛应用的数字递推PID控制算法


## 采样周期的选择

香农(Shannon)采样定律: 为不失真地复现信号的变化, 采样频率至少应大于或等于连续信号最高频率分量的二倍. 根据采样定律可以确定采样周期的上限值, 实际采采样周期的选择还要受到多方面因素的影响, 不同的系统采样周期应根据具体情况来选择.

采样周期的选择, 通常按照过程特性与干扰大小适当来选取采洋周期: 即对于响应快(如流量/压力), 波动大, 易受干扰的过程, 应选取较短的采样周期; 反之, 当过程响应慢(如温度, 成份), 滞后大时可选取较长的采样周期.

采样周期的选取应与 PID 参数的整定进行综合考虑, 采样周期应远小于过程的扰动信号的周期, 在执行器的响应速度比较慢时, 过小的采样周期将失去意义, 因此可适当运选大一点. 在计算机运算速度允许的条件下, 采样周期短, 则控制品质好. 当过程的纯滞后时间较长时, 一般选取采样周期为纯滞后时间的 1/4 ~ 1/8